<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.19.2
Pandas:  1.1.3


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [3]:
# wrong:
#bikes = pd.read_table('bikeshare.csv', header = None)
#bikes = pd.read_csv('bikeshare.csv')

bikes.head()
#print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1, delimiter = ',')
print(bikes.head())
#print()

# wrong:
#bikes = pd.read_table('bikeshare.csv', header = 0)
#print(bikes.head())


NameError: name 'bikes' is not defined

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

In [4]:
# wrong:
bikes = pd.read_csv('bikeshare.csv', header = None)
print(bikes.head())
#print()

        0           1       2   3     4   5        6        7           8   \
0  instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday   
1        1  2011-01-01       1   0     1   0        0        6           0   
2        2  2011-01-01       1   0     1   1        0        6           0   
3        3  2011-01-01       1   0     1   2        0        6           0   
4        4  2011-01-01       1   0     1   3        0        6           0   

           9     10      11    12         13      14          15   16  
0  weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
1           1  0.24  0.2879  0.81          0       3          13   16  
2           1  0.22  0.2727   0.8          0       8          32   40  
3           1  0.22  0.2727   0.8          0       5          27   32  
4           1  0.24  0.2879  0.75          0       3          10   13  


Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [5]:
#ANSWER:
bikes = pd.read_csv('bikeshare.csv')

bikes.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [ ]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [ ]:
df = pd.read_excel('Iris.xls', sheet_name = 'Data')
df.head()

So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [ ]:
#ANSWER
#remove species_no

df = pd.read_excel('Iris.xls', sheet_name = 'Data')
df.drop(columns=['Species_No'])

### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [ ]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [ ]:
url = 'https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/'

df = pd.read_html(url)
df

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [ ]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [ ]:
#ANSWER
bikes = pd.read_csv('bikeshare.csv')
bikes.shape

What are the column names in `bikes`?

In [ ]:
#ANSWER
bikes = pd.read_csv('bikeshare.csv')
list(bikes.columns)

What are the data types of these columns?

In [ ]:
#ANSWER
bikes = pd.read_csv('bikeshare.csv')
bikes.dtypes
#d.dtype how to find data type

What is the (row) index for this DataFrame?

In [ ]:
#ANSWER
#.loc
bikes = pd.read_csv('bikeshare.csv')
bikes.loc[1]

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [ ]:
#ANSWER
bikes = pd.read_csv('bikeshare.csv')
bikes['season']

In [ ]:
bikes = pd.read_csv('bikeshare.csv')
bikes[['season']]

How would we use object notation to show the first 4 rows of `atemp`?

In [12]:
#ANSWER
bikes = pd.read_csv('bikeshare.csv')
bikes['atemp'].head()

0    0.2879
1    0.2727
2    0.2727
3    0.2879
4    0.2879
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [13]:
bikes = pd.read_csv('bikeshare.csv')
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [14]:
#ANSWER
bikes = pd.read_csv('bikeshare.csv')
bikes.iloc[0]

instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.24
atemp             0.2879
hum                 0.81
windspeed              0
casual                 3
registered            13
cnt                   16
Name: 0, dtype: object

What is the correct way to access the 2nd column of the DataFrame by its index?

In [15]:
#ANSWER
bikes = pd.read_csv('bikeshare.csv')
bikes.iloc[1]

instant                2
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     1
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.22
atemp             0.2727
hum                  0.8
windspeed              0
casual                 8
registered            32
cnt                   40
Name: 1, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER: #detect missing value 

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [ ]:
bikes.isnull().head()

However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [ ]:
#ANSWER

bikes.isnull().sum().sum()

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [ ]:
#ANSWER

#bikes[bikes.columns[bikes.isnull().any()]]
bikes.columns[(bikes.isna().any())].tolist()
#bikes.isnull().sum()

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid. value that is undefine. like N/A

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [ ]:
#ANSWER
x='3'
try:
  print(x/3)
except:
  print("Something went wrong")
finally:
  print("The 'try except' is finished")



Apply the Pandas `isna` function to the following data objects:

In [ ]:
x = 2.3
y = np.nan
print(x, y)

In [ ]:
#ANSWER
pd.isna(y)

In [ ]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

In [ ]:
#ANSWER
pd.isna(array)

How is the pandas I/O parameter `na_values` used?

? ANSWER:Now the na_values parameter is used to tell pandas they consider “not available” as NaN value and print NaN at the place of “not available 

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [16]:
#ANSWER
# importing pandas package
import pandas as pd
  
# making data frame from csv file
bikes = pd.read_csv('bikeshare.csv')
  
# storing unique value in a variable
unique_value = bikes["season"].nunique()
  
# printing value
print(unique_value)

4


### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [ ]:
#ANSWER
print((bikes.instant.max())-(bikes.instant.min()))
#print((bikes.dteday.max())-(bikes.dteday.min()))
print((bikes.windspeed.max())-(bikes.windspeed.min()))
print(bikes.dteday.max())
print(bikes.dteday.min())

In [ ]:
u = bikes.select_dtypes(include=[np.number])
# U = u.apply(np.ptp, axis=0)
U = u.max() - u.min()

v = bikes.select_dtypes(include=[object])
V = v.max() + ' - ' + v.min()

print (U.append(V))

Compute and print the overall minimum and maximum of the numeric data columns:

In [ ]:
bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
bikes_min, bikes_max


In [ ]:
u = bikes.select_dtypes(include=[np.number])
# U = u.apply(np.ptp, axis=0)
bikes_max = u.max()
bikes_min = u.min()



print (bikes_max)
print(bikes_min)

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [ ]:
bikes['atemp'].quantile(0.5)

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [ ]:
#ANSWER
bikes['atemp'].quantile([0.1,0.25,0.5,0.75,0.9])

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [ ]:
type(bikes['atemp'])

In [ ]:
bikes.sample(5)

Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [ ]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [ ]:
#ANSWER
bikes.sample(5)

Here is a random sample of `atemp_level`:

In [ ]:
atemp_level.sample(5)          

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [ ]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)
bikes.sample(5)

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [ ]:
#ANSWER
bikes["category"] = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
#(bikes.category == 'mild',bikes.season == 2).sum()
#bikes.sample(5)
bikes[(bikes['category'] == 'mild')& (bikes['season'] == 2)].shape[0]


#bikes.groupby(['season']).category.value_counts().unstack(fill_value=0).loc[:,'mild']

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [ ]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

print(a.shape)
print(b.shape)
print(c.shape)
print(d.shape)
print(e.shape)

In [ ]:
# Cleaning Data

## Load Data

#Load rock.csv and clean the dataset.

rock = pd.read_csv('rock.csv')

In [8]:
rock = pd.read_csv('rock.csv')
rock.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


## Check Column Names

Check column names and clean.

In [27]:
rock.columns = ['song','artist','release year',
                     'combined','first','year', 'playcount', 'fg']
rock.head()

,song,artist,release year,combined,first,year,playcount,fg
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


## Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [56]:

rock['release year'].fillna(0)
rock.head()
#rock.dtypes

,song,artist,release year,combined,first,year,playcount,fg
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [74]:
#rock.astype({"release year": int})
rock['release year']=rock['release year'].fillna(0)

#rock["release year"].replace({"SONGFACTS.COM": '0'}, inplace=True)
rock['release year'].unique()
#rock.head()
rock['release year'] = rock['release year'].astype(int)
#pd.to_numeric(rock['release year'])
rock.dtypes

song            object
artist          object
release year     int64
combined        object
first            int64
year             int64
playcount        int64
fg               int64
dtype: object

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [78]:
u = rock.select_dtypes(include=[np.number])
# U = u.apply(np.ptp, axis=0)
rock_max = u.max()
rock_min = u.min()
print (rock_max)
print ('-')
print (rock_min)

release year    2014
first              1
year               1
playcount        142
fg               142
dtype: int64
-
release year    0
first           1
year            0
playcount       0
fg              0
dtype: int64


## Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [101]:
rock.head()
row_num = 4
rock.iloc[row_num,[0,1,2]]
rock.iloc[row_num,2]

if rock.iloc[row_num,2] < 1970:
    print("old song")
else: 
    print("new song")
    

def detail(row_num):
    print(rock.iloc[row_num,[0,1,2]])
    
    if rock.iloc[row_num,2] < 1970:
        print("old song")
    else: 
        print("new song")
    

    
    
    
    



new song


In [102]:
detail(5)

song              Kryptonite
artist          3 Doors Down
release year            2000
Name: 5, dtype: object
new song


In [111]:
rock.dtypes

rock.apply(pd.to_numeric, downcast="float",errors ='coerce')

,song,artist,release year,combined,first,year,playcount,fg
0,NaN,NaN,1982.0,NaN,1.0,1.0,82.0,82.0
1,NaN,NaN,0.0,NaN,1.0,0.0,3.0,0.0
2,NaN,NaN,1981.0,NaN,1.0,1.0,85.0,85.0
3,NaN,NaN,1980.0,NaN,1.0,1.0,18.0,18.0
4,NaN,NaN,1975.0,NaN,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
2225,NaN,NaN,0.0,NaN,1.0,0.0,1.0,0.0
2226,NaN,NaN,1981.0,NaN,1.0,1.0,32.0,32.0
2227,NaN,NaN,1975.0,NaN,1.0,1.0,109.0,109.0
2228,NaN,NaN,1983.0,NaN,1.0,1.0,1.0,1.0


In [112]:
rock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   song          2230 non-null   object
 1   artist        2230 non-null   object
 2   release year  2230 non-null   int64 
 3   combined      2230 non-null   object
 4   first         2230 non-null   int64 
 5   year          2230 non-null   int64 
 6   playcount     2230 non-null   int64 
 7   fg            2230 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 139.5+ KB


## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [121]:
def converter(df):
    #df.apply(pd.to_numeric, downcast="float",errors ='coerce')
    new_df=df.apply(pd.to_numeric, downcast="float",errors ='coerce')
    return new_df

## Apply these functions to your dataset

In [126]:
new_rock = converter(rock)
print(new_rock)

      song  artist  release year  combined  first  year  playcount     fg
0      NaN     NaN        1982.0       NaN    1.0   1.0       82.0   82.0
1      NaN     NaN           0.0       NaN    1.0   0.0        3.0    0.0
2      NaN     NaN        1981.0       NaN    1.0   1.0       85.0   85.0
3      NaN     NaN        1980.0       NaN    1.0   1.0       18.0   18.0
4      NaN     NaN        1975.0       NaN    1.0   1.0        1.0    1.0
...    ...     ...           ...       ...    ...   ...        ...    ...
2225   NaN     NaN           0.0       NaN    1.0   0.0        1.0    0.0
2226   NaN     NaN        1981.0       NaN    1.0   1.0       32.0   32.0
2227   NaN     NaN        1975.0       NaN    1.0   1.0      109.0  109.0
2228   NaN     NaN        1983.0       NaN    1.0   1.0        1.0    1.0
2229   NaN     NaN        1973.0       NaN    1.0   1.0        2.0    2.0

[2230 rows x 8 columns]


NameError: name 'new_df' is not defined

## Describe the new float-only DataFrame.

In [127]:
new_rock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   song          2 non-null      float32
 1   artist        0 non-null      float32
 2   release year  2230 non-null   float32
 3   combined      0 non-null      float32
 4   first         2230 non-null   float32
 5   year          2230 non-null   float32
 6   playcount     2230 non-null   float32
 7   fg            2230 non-null   float32
dtypes: float32(8)
memory usage: 69.8 KB


>




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



